In [1]:
import os
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler



In [2]:
class PRPDMetrics():

    def __init__(self, PRPD: np.ndarray):
        """
        Inicializa a classe com um array.

        Args:
            PRPD(ndarray): Array NumPy contendo valores de amplitude e fase.
        """

        if not isinstance(PRPD, np.ndarray): # Validações de entrada
            raise TypeError("PRPD deve ser um array NumPy.")
        if PRPD.size == 0:
            raise ValueError("PRPD não pode ser vazio.")
        
        self.PRPD = PRPD   
        self._minimum = None # prefixo _ indica que são variáveis internas (convenção Python)
        self._maximum = None
        self._elements = None
        self._mean = None
        self._Firstquartile = None
        self._Secondquartile = None
        self._Thirdquartile = None
        self._Assymetry = None
        self._Kurtosis = None
        self._dataframe = None

    @property # permite acessar o método como se fosse um atributo (sem parênteses)
    def minimum(self) -> float:
        """Calcula e retorna o valor minimo dos vetores de amplitudes e fases"""
        if self._minimum is None:
            self._minimum = np.min(self.PRPD, axis=1)
        return self._minimum
    
    @property
    def maximum(self) -> float:
        """Calcula e retorna o valor maximo dos vetores de amplitude e fase"""
        if self._maximum is None:
            self._maximum = np.max(self.PRPD, axis=1)
        return self._maximum
    
    @property
    def elements(self) -> float:
        """Calcula e retorna a quantidade de pulsos"""
        if self._elements is None:
            self._elements = self.PRPD.shape[1]
        return self._elements
    
    @property
    def mean(self) -> float:
        """Calcula e retorna a media dos amplitudes e fases"""
        if self._mean is None:
            self._mean = np.mean(self.PRPD, axis=1)
        return self._mean
    
    @property
    def firstquartile(self) -> float:
        """Calcula e retorna o primeiro quartil das amplitudes e fases"""
        if self._Firstquartile is None:
            self._Firstquartile = np.percentile(self.PRPD, 25, axis=1)
        return self._Firstquartile
    
    @property
    def secondquartile(self) -> float:
        """Calcula e retorna o segundo quartil das amplitudes e fases"""
        if self._Secondquartile is None:
            self._Secondquartile = np.percentile(self.PRPD, 50, axis=1)
        return self._Secondquartile
    
    @property
    def thirdquartile(self) -> float:
        """Calcula e retorna o terceiro quartil das amplitudes e fases"""
        if self._Thirdquartile is None:
            self._Thirdquartile = np.percentile(self.PRPD, 75, axis=1)
        return self._Thirdquartile
    
    @property
    def assymetry(self) -> float:
        """Calcula e retorna o assimetria das amplitudes e fases"""
        if self._Assymetry is None:
            self._Assymetry = scipy.stats.skew(self.PRPD, axis=1)
        return self._Assymetry
    
    @property
    def kurtosis(self) -> float:
        """Calcula e retorna a curtose das amplitudes e fases"""
        if self._Kurtosis is None:
            self._Kurtosis = scipy.stats.kurtosis(self.PRPD, axis=1)
        return self._Kurtosis  
    
    @property
    def dataframe(self) -> pd.DataFrame:
        """Retorna um DataFrame com todas as métricas calculadas"""
        if self._dataframe is None:
            self._dataframe = pd.DataFrame({
                'Minimum': self.minimum,
                'Maximum': self.maximum,
                'Elements': self.elements,
                'Mean': self.mean,
                'First Quartile': self.firstquartile,
                'Second Quartile': self.secondquartile,
                'Third Quartile': self.thirdquartile,
                'Assymetry': self.assymetry,
                'Kurtosis': self.kurtosis}, 
                index = ['Amplitude', 'Fase'])
        return self._dataframe
            
        
    

In [3]:
def PRPDExtraction(Diretório_entrada: str, Diretório_saida: str) -> None:
    """
    Extrai as características de um arquivo .mat contendo dados PRPD e salva em um arquivo .csv.

    Args:
        Diretório_entrada(str): Diretório onde estão os arquivos .mat.
        Diretório_saida(str): Diretório onde serão salvos os arquivos .csv.
    """

    #Lista os arquivos no diretório de entrada e define um DataFrame vazio
    Arquivos = os.listdir(Diretório_entrada)
    df = pd.DataFrame()

    for i in range(len(Arquivos)):
        if Arquivos[i].endswith(".mat"):

            #Carrega os dados do arquivo .mat
            dados = scipy.io.loadmat(os.path.join(Diretório_entrada, Arquivos[i]))

            #Cria um DataFrame com os dados do arquivo .mat
            df = pd.DataFrame(list(dados.values())[-1])

            # Criar a coluna de classe
            df.loc['classe'] = ['positivo' if valor <= 180 else 'negativo' for valor in df.iloc[1, :]]

            # Filtrar os elementos do semi-ciclo positivo
            dados_sp = df.loc[:, df.loc['classe'] == 'positivo'].drop('classe').to_numpy()[:2, :]
            dados_sp = dados_sp.astype(float)  # Garantir que todos os elementos sejam floats

            # Filtrar elementos do semi-ciclo negativo
            dados_sn = df.loc[:, df.loc['classe'] == 'negativo'].drop('classe').to_numpy()[:2, :]
            dados_sn = dados_sn.astype(float)  # Garantir que todos os elementos sejam floats

            # Filtrar os elementos do semi-ciclo positivo
            dados_sp = df.loc[:, df.loc['classe'] == 'positivo'].drop('classe').to_numpy()[:2, :]
            dados_sp = dados_sp.astype(float)  # Garantir que todos os elementos sejam floats

            # Criar a coluna de classe
            df.loc['classe'] = ['positivo' if valor <= 180 else 'negativo' for valor in df.iloc[1, :]]

            # Filtrar elementos do semi-ciclo negativo
            dados_sn = df.loc[:, df.loc['classe'] == 'negativo'].drop('classe').to_numpy()[:2, :]
            dados_sn = dados_sn.astype(float)  # Garantir que todos os elementos sejam floats
            
            #Normalizar os vetores de amplitudes dos semi-ciclos:
            dados_sn[0,:] = dados_sn[0,:]/max(dados_sn[0,:])
            dados_sp[0,:] = dados_sp[0,:]/max(dados_sp[0,:])

            #Aplicando a classe metrics para extrair os atributos:
            metrics_sn = PRPDMetrics(dados_sn)
            metrics_sp = PRPDMetrics(dados_sp)

            #Salvando o Dataframe com as features em um csv
            nome_arquivo_sp = 'PRPDfeatures_sp_' + Arquivos[i] + '.csv'
            nome_arquivo_sn = 'PRPDfeatures_sn_' + Arquivos[i] + '.csv'
            Diretório_saida_sp = os.path.join(Diretório_saida, nome_arquivo_sp)
            Diretório_saida_sn = os.path.join(Diretório_saida, nome_arquivo_sn)

            df_sp = metrics_sp.dataframe.T
            df_sn = metrics_sn.dataframe.T
            

            df_sp.to_csv(Diretório_saida_sp, index = True)
            df_sn.to_csv(Diretório_saida_sn, index = True)

        else:
            continue

    


Diretório_entrada = r"C:\Users\boave\OneDrive\Desktop\Pasta Pessoal - Boaventura Filho\PIBIC\Dados"
Diretório_saida = r"C:\Users\boave\OneDrive\Desktop\Pasta Pessoal - Boaventura Filho\PIBIC\Resultados\Barra_6kv"

PRPDExtraction(Diretório_entrada, Diretório_saida,)

In [4]:
# Validando os arquivos salvos
arquivos_saida = os.listdir(Diretório_saida)

for arquivo in arquivos_saida:
    arquivo
    df = pd.read_csv(os.path.join(Diretório_saida, arquivo), index_col = 0)
    print(df)



                 Amplitude        Fase
Minimum           0.614595  202.483736
Maximum           1.000000  265.587097
Elements         23.000000   23.000000
Mean              0.822857  226.605936
First Quartile    0.735483  209.404984
Second Quartile   0.815676  227.669605
Third Quartile    0.912027  236.089914
Assymetry        -0.125541    0.500099
Kurtosis         -0.972562   -0.651366
                   Amplitude         Fase
Minimum             0.300666   180.014284
Maximum             1.000000   359.951346
Elements         1986.000000  1986.000000
Mean                0.439838   270.072205
First Quartile      0.377711   227.684901
Second Quartile     0.415246   269.688382
Third Quartile      0.498218   312.763892
Assymetry           1.525359     0.008389
Kurtosis            4.355878    -1.116069
                 Amplitude       Fase
Minimum           0.659698  18.125502
Maximum           1.000000  76.367047
Elements         29.000000  29.000000
Mean              0.827449  43.576051
